In [1]:
import pandas as pd
import openai
import os
from annotation_utils import get_val_scores, make_transcript
from AptTagger import AptTagger
from TruthLabeler import TruthLabeler

openai.api_key_path = "/home/alex/.personal/openAIkey"  # read protected to my 174 account

/mnt/ssd-2/spar/alexm/miniconda3/envs/dlkb/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# val_scores_df = get_val_scores()
# val_df = pd.read_json("data/label-validation/transcripts_combined.json")
# val_df.index = val_df["message_id"]
# val_df = val_df.join(val_scores_df, how="inner")
# main_df = val_df

# apt_tagger = AptTagger(
#     model_name="data/apt-tagger-deberta-xlarge-mnli-s63day9p",
#     device="cuda"
# )

# main_df["tag_mask"] = apt_tagger.tag(main_df["assistant_text"])
# main_df["annotated_assistant_text"] = main_df.apply(lambda x: apt_tagger.annotate(x["assistant_text"], x["tag_mask"]), axis=1)
# main_df["annotated_transcript"] = main_df.apply(make_transcript, axis=1, args=(True,))
# main_df.head()

In [55]:
annotated_val_df = pd.read_json("data/label-validation/annotated_transcripts_combined.json")
main_df = annotated_val_df

In [ ]:
from Retriever import Retriever
retriever = Retriever(
    use_IVF=True,
    encoder_name="BAAI/bge-small-en",
    pretrained_index_path='retrieval/20230601.en.wiki_embeddings_bge-small-en_IVF.index',
)

In [72]:
truth_labeler = TruthLabeler(
                retriever,
                model_name="gpt-3.5-turbo",
                temperature=1,
                n_samples=5,
                uncertainty_bias=0,
                na_bias=0,
)

In [56]:
out_df = truth_labeler.label(
    ids=main_df["message_id"],
    annotated_transcripts=main_df["annotated_transcript"], 
    retrieval_queries=main_df["assistant_text"],
    n_threads=10)
# NOTE: THERE ARE DUPLICATE IDS IN THE OUTPUT DF WHEN USING RETRIEVER SO THAT WE CAN COMPARE THE RESULTS
# TODO: Change this once I've finished validating the retriever
out_df_retr = out_df[out_df["used_retrieval"]]
retr_ids = set(out_df_retr["message_id"])
out_df_counter_retr = out_df[~((out_df["message_id"].isin(retr_ids)) & out_df["used_retrieval"])]
# remove all the rows with message_id in retr_ids but with used_retrieval==False
out_df = out_df[~(out_df["message_id"].isin(retr_ids) & (~out_df["used_retrieval"]))]

SKIPPING: scores must be one of ['N/A', 'Determinable - Certainly True', 'Determinable - True', 'Determinable - Uncertain', 'Determinable - False', 'Determinable - Certainly False'], but found ['[[1]] Determinable - True', '[[2]] Determinable - Uncertain', '[[3]] Determinable - Uncertain', '[[4]] Determinable - Uncertain', 'Wikipedia cross-check needed? No']
SKIPPING: 1 scores found, but 4 annotations were expected.
SKIPPING: scores must be one of ['N/A', 'Determinable - Certainly True', 'Determinable - True', 'Determinable - Uncertain', 'Determinable - False', 'Determinable - Certainly False'], but found ['[[1]] Determinable - True', '[[2]] Determinable - True', '[[3]] Determinable - Uncertain', '[[4]] Determinable - True', '[[5]] Determinable - Uncertain', '[[6]] Determinable - True', 'Wikipedia cross-check needed? No']
SKIPPING: scores must be one of ['N/A', 'Determinable - Certainly True', 'Determinable - True', 'Determinable - Uncertain', 'Determinable - False', 'Determinable - Ce

In [60]:
main_df.index = main_df["message_id"]
labeled_df = out_df.copy()
labeled_df.index = out_df["message_id"]
labeled_df = labeled_df.drop(columns=["message_id", "annotated_transcript"])
labeled_df = labeled_df.join(main_df, how="left")
print("length:", len(labeled_df))
labeled_df.head()

length: 19


,input,retrieval_query,responses,prompt_tokens,completion_tokens,dollars,pred_scores,p_apts,p_trues,ann_count,...,model,assistant_text,user_text,message_id,prev_messages,rank,scores,tag_mask,annotated_assistant_text,annotated_transcript
message_id,,,,,,,,,,,,,,,,,,,,,
6a108ea6-4c2e-4250-9969-7206c11fe6aa,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,Authoritarianism can sometimes help prevent so...,[[[1]] Score: Determinable - True\n[[2]] Score...,471,201,0.001108,"[(Determinable - True, Determinable - Uncertai...","[0.7666666666666666, 0.5333333333333333, 0.3, ...","[0.7000000000000001, 0.6, 0.5, 0.5]",4,...,keyfan/bloomz-rlhf(max_length=512;temperature=...,Authoritarianism can sometimes help prevent so...,Is authoritarianism sufficient to keep countri...,6a108ea6-4c2e-4250-9969-7206c11fe6aa,[Is authoritarianism sufficient to keep countr...,NaN,"[Certainly True, Certainly True, Certainly Tru...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Authoritarianism can sometimes help prevent so...,USER: Is authoritarianism sufficient to keep c...
c1f98662-f2a3-4cd2-8d56-03532df73ada,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,SpaceX is an American entrepreneurial aerospac...,[[[1]] Score: Determinable - True\n[[2]] Score...,504,323,0.001402,"[(Determinable - True, Determinable - True, De...","[1.0, 1.0, 0.825, 0.825, 0.47500000000000003, ...","[0.8500000000000001, 0.8, 0.7250000000000001, ...",6,...,keyfan/bloomz-rlhf(max_length=512;temperature=...,SpaceX is an American entrepreneurial aerospac...,What is SpaceX?,c1f98662-f2a3-4cd2-8d56-03532df73ada,[What is SpaceX?],NaN,"[Certainly True, False, Certainly True, Certai...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",SpaceX is an American entrepreneurial aerospac...,USER: What is SpaceX?\n\nASSISTANT: SpaceX is ...
a9bb02bd-c4da-4f47-a55d-53c94726cc71,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,If animals could speak like humans and convey ...,[[[1]] Score: Determinable - True\n[[2]] Score...,627,390,0.001721,"[(Determinable - True, Determinable - True, De...","[0.86, 0.86, 0.3, 0.44000000000000006, 0.43999...","[0.74, 0.74, 0.5, 0.44000000000000006, 0.55999...",7,...,keyfan/bloomz-rlhf(max_length=512;temperature=...,If animals could speak like humans and convey ...,Imagine a world where animals can talk. How do...,a9bb02bd-c4da-4f47-a55d-53c94726cc71,[Imagine a world where animals can talk. How d...,NaN,"[Certainly True, True, Certainly False, False,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",If animals could speak like humans and convey ...,USER: Imagine a world where animals can talk. ...
fd9fa103-6988-4d4c-b6de-2e7a0ac8b5a9,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,Joseph Scaliger (1557-1636) was an Italian sch...,[[[1]] Score: Determinable - Uncertain\n[[2]] ...,560,438,0.001716,"[(Determinable - Uncertain, Determinable - Cer...","[0.86, 1.0, 1.0, 1.0, 0.44000000000000006, 0.8...","[0.7799999999999999, 0.8400000000000001, 0.8, ...",8,...,keyfan/bloomz-rlhf(max_length=512;temperature=...,Joseph Scaliger (1557-1636) was an Italian sch...,Describe Joseph Scaliger and his contributions...,fd9fa103-6988-4d4c-b6de-2e7a0ac8b5a9,[Describe Joseph Scaliger and his contribution...,NaN,"[Certainly False, True, True, Certainly True, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Joseph Scaliger (1557-1636) was an Italian sch...,USER: Describe Joseph Scaliger and his contrib...
3a663153-5eeb-4b42-8ab0-ba8bae6c8ab6,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,The potential obstacles faced by human explore...,[[[1]] Score: N/A\n[[2]] Score: Determinable -...,629,422,0.001788,"[(N/A, Determinable - Uncertain, Determinable ...","[0.575, 0.825, 0.6499999999999999, 1.0, 0.65, ...","[0.65, 0.525, 0.65, 0.5, 0.65, 0.8, 0.5, 0.850...",8,...,vihangd/smartplat-3b-v1(max_length=512;tempera...,The potential obstacles faced by human explore...,If humans will be able to go to Mars and live ...,3a663153-5eeb-4b42-8ab0-ba8bae6c8ab6,[If humans wil

In [64]:
from datetime import datetime
# save results
dt = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
labeled_df.to_json(f"data/label-validation/dataframes/{dt}.json", orient="records")

In [70]:
import numpy as np

cat_preds = np.concatenate(labeled_df["p_trues"].values)
cat_p_apt = np.concatenate(labeled_df["p_apts"].values)
cat_gt = np.concatenate(labeled_df["scores"].values)
expected_anns = np.sum(labeled_df["ann_count"].values)

prop_retrieval = labeled_df["used_retrieval"].mean()

# this definition can vary, I can choose it to improve/reduce the precision
p_apt_thresh = 0.7
p_true_thresh = 0.75
apt = cat_p_apt >= p_apt_thresh
pred_true = (cat_preds >= p_true_thresh) & apt
pred_false = (cat_preds <= (1 - p_true_thresh)) & apt

prec_true = (cat_gt[pred_true] == "Certainly True").mean()
prec_false = (cat_gt[pred_false] == "Certainly False").mean()

anti_prec_true = (cat_gt[pred_true] == "Certainly False").mean()
anti_prec_false = (cat_gt[pred_false] == "Certainly True").mean()

prop_true = pred_true.sum() / expected_anns
prop_false = pred_false.sum() / expected_anns
proportion_apt = prop_true + prop_false
print(f"Proportion of predicted true statements that are actually true: {prec_true}")
print(f"Proportion of predicted true statements that are actually false: {anti_prec_true}")
print(f"Proportion of predicted false statements that are actually false: {prec_false}")
print(f"Proportion of predicted false statements that are actually true: {anti_prec_false}")
print(f"Proportion of statements predicted true: {prop_true}")
print(f"Proportion of statements predicted false: {prop_false}")
print(f"Proportion of statements predicted truth-apt: {proportion_apt}")
print(f"Proportion of transcripts where retrieval was used: {prop_retrieval}")

Proportion of predicted true statements that are actually true: 0.3333333333333333
Proportion of predicted true statements that are actually false: 0.06060606060606061
Proportion of predicted false statements that are actually false: 0.6666666666666666
Proportion of predicted false statements that are actually true: 0.0
Proportion of statements predicted true: 0.4024390243902439
Proportion of statements predicted false: 0.018292682926829267
Proportion of statements predicted truth-apt: 0.4207317073170732
Proportion of transcripts where retrieval was used: 0.05263157894736842


In [ ]:
import json

with open("data/label-validation/logs.json", "r") as f:
    avg_scores = json.load(f)

score_object = {
    **truth_labeler.get_metadata(),
    "p_apt_thresh": p_apt_thresh,
    "p_true_thresh": p_true_thresh,
    "proportion_apt": proportion_apt,
    "datetime": dt,
    "prec_true": prec_true,
    "prec_false": prec_false if not np.isnan(prec_false) else None,
    "anti_prec_true": anti_prec_true,
    "anti_prec_false": anti_prec_false if not np.isnan(anti_prec_false) else None,
    "prop_retrieval": prop_retrieval,
    "prop_pred_true": prop_true,
    "prop_pred_false": prop_false,
}
avg_scores.append(score_object)
with open("data/label-validation/logs.json", "w") as f:
    json.dump(avg_scores, f, indent=2)
score_object

In [ ]:
# TODO: save scores to data/truth_label_scores.json
# TODO: test that the pipeline records accuracy correctly``
